In [1]:
import numpy as np
import cv2
import face_recognition
import imutils
import pickle

In [2]:
videofile = 'team-rotated.mp4'
path_prefix = 'C:/Users/jshru_000/bootcamp1/FINAL/faces/'
pickle_path = path_prefix+'facesmeta.pkl'

In [3]:
data = []
with open(pickle_path, 'rb') as f:
    try:
        while True:
            data.append(pickle.load(f))
    except EOFError:
        pass

In [4]:
encdb = []
names = []
fnames = list(data[0].keys())
for fname in fnames:
    encdb.append(data[0][fname]['encoding'])
    names.append(data[0][fname]['name'])

In [5]:
def resizeframe(image, width=500):
    w = float(width)
    r = w / image.shape[1]
    dim = (width, int(image.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    xr = dim[0]/float(image.shape[1])
    yr = dim[1]/float(image.shape[0])
    return resized

In [6]:
def draw_str(dst, x, y, s, right=0):
    fontFace = cv2.FONT_HERSHEY_PLAIN
    fontScale = 1.0
    thickness = 2
    txtsize = cv2.getTextSize(s, fontFace, fontScale, thickness)
    text_width = txtsize[0][0]
    if right == 0:
        cv2.putText(dst, s, (x+1, y+1), fontFace, fontScale, (0, 0, 0), thickness = thickness, lineType=cv2.LINE_AA)
        cv2.putText(dst, s, (x, y), fontFace, fontScale, (0, 0, 255), lineType=cv2.LINE_AA)
    else:
        wi = right - x
        if text_width > wi:
            scalefactor = wi/text_width
            fontScale *= scalefactor
        cv2.putText(dst, s, (x+1, y+1), fontFace, fontScale, (0, 0, 0), thickness = thickness, lineType=cv2.LINE_AA)
        cv2.putText(dst, s, (x, y), fontFace, fontScale, (0, 0, 255), lineType=cv2.LINE_AA)
        

In [7]:
def get_faces(frame):
    newframe = frame.copy()
    model = 'hog' # 'cnn'
    metadata = {}
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = imutils.resize(frame, width=750)
    r = frame.shape[1] / float(rgb.shape[1])
    boxes = face_recognition.face_locations(rgb,model=model)
    if not boxes == []:
        encodings = face_recognition.face_encodings(rgb, boxes)
        assert len(encodings) == len(boxes)
        for i, (top, right, bottom, left) in enumerate(boxes):
            top = int(top * r)
            right = int(right * r)
            bottom = int(bottom * r)
            left = int(left * r)
            cv2.rectangle(newframe, (left, top), (right, bottom), (0, 255, 0), 2)
            #thisFace = frame[top:bottom,left:right]
            name = search_facedb(encodings[i])
            draw_str(newframe,left,top,name, right=right)
    else:
        draw_str(newframe,20,20,'No face identified')
    return(newframe)

In [8]:
def search_facedb(enc):
    tolerance = 0.5
    matches = face_recognition.compare_faces(encdb,enc,tolerance=tolerance)
    if True in matches:
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        for i in matchedIdxs:
            name = names[i]
            counts[name] = counts.get(name, 0) + 1
        name = max(counts, key=counts.get)
    else:
        name='unknown'
    return(name)

In [9]:
cap1 = cv2.VideoCapture(videofile)
w1 = int(cap1.get(3))
h1 = int(cap1.get(4))
face_counter = 0
flag = True
frameno = 0
frameinterval = 1 #10

while cap1.isOpened():
    ret1, frame1 = cap1.read()
    frameno += 1
    if not ((frameno % frameinterval) == 0):
        continue
    if ret1 :
        frame1 = resizeframe(frame1)
        frame = frame1.copy()
        cv2.imshow('Frames- pause p; resume r; quit q',frame)
        if cv2.waitKey(1) & 0xFF == ord('p'): #pause
            while (True):
                if flag:
                    newframe = get_faces(frame)
                    cv2.imshow('Frames- pause p; resume r; quit q',newframe)
                    flag = False
                if cv2.waitKey(1) & 0xFF == ord('r'): #resume
                    flag = True
                    break
        if cv2.waitKey(10) & 0xFF == ord('q'): #quit
            break
    else:
        break
cap1.release()
cv2.destroyAllWindows()